In [1]:
import requests 

from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from time import sleep
import pandas as pd
import numpy as np
import datetime
import os

# Inicializar parametros iniciales

In [2]:
# PONER USER-AGENT PROPIO:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
# Options:
opts = Options()
opts.add_argument("--disable-extensions")
opts.add_argument(f"User-Agent={user_agent}")

# URL de los empleos en cada página

In [3]:
# Defino lista url empleos:
url_empleos = []

In [9]:
pagina = 1
url_infojobs = f"https://www.infojobs.net/ofertas-trabajo/informatica-telecomunicaciones?keyword=&categoryIds=150&segmentId=&page={pagina}&sortBy=PUBLICATION_DATE&onlyForeignCountry=false&sinceDate=ANY"
url_infojobs_nuevas = f"https://www.infojobs.net/jobsearch/search-results/list.xhtml?keyword=&categoryIds=150&segmentId=&page={pagina}&sortBy=PUBLICATION_DATE&onlyForeignCountry=false&sinceDate=_24_HOURS"

# Initialize:
browser = webdriver.Chrome(options=opts)
browser.maximize_window()
browser.get(url_infojobs_nuevas)

##########Captcha##########
sleep(15)
###########################

# Acepto cookies
browser.find_elements(By.ID, "didomi-notice-agree-button")[0].click()

while True:
    
    # Saco url empleos:
    soup_urls = BeautifulSoup(browser.page_source, "html.parser")

    try:
        urls = soup_urls.find_all("div", class_= "ij-OfferCardContent-description")
    except:
        break

    for url in urls:
        oferta_url = url.find("h2").find("a")["href"].replace("//", "")
        url_empleos.append(oferta_url)

    pagina += 1
    browser.find_elements(By.CLASS_NAME, "sui-MoleculePagination-item")[-1].click()
    sleep(2)
    
    lim_paginas = len(browser.find_elements(By.CLASS_NAME, "sui-MoleculePagination-item"))
    
    if pagina > 1:
        if lim_paginas < 7:
            break

In [10]:
url_empleos_final = ["https://" + empleo for empleo in url_empleos]

In [3]:
# df_urls_empleo = pd.DataFrame(url_empleos_final, columns= ["url"])
# df_urls_empleo.to_csv("url_empleos.csv", index= False, sep = ",")

# Saco la info de cada empleo:

In [4]:
# En caso de haber guardado csv de urls:
# df_urls_empleo = pd.read_csv("url_empleos.csv")
# url_empleos_final = [url[0] for url in df_urls_empleo.values.tolist()]

In [4]:
datos_ofertas_empleo = []
contador = 0

In [ ]:
portal = "infojobs"
fecha_scrapeo = datetime.datetime.now().date()

for empleo in url_empleos_final[contador:]:
    
    if contador == 0:
        browser = webdriver.Chrome(options=opts)
        browser.maximize_window()
        browser.get(empleo)
        
        ##########Captcha##########
        sleep(15)
        ###########################

        # Acepto cookies
        browser.find_elements(By.ID, "didomi-notice-agree-button")[0].click()
        sleep(1)
        
    else:
        browser.get(empleo)
        sleep(3)
        
    # Saco información relevante de la oferta:    
    soup_oferta = BeautifulSoup(browser.page_source, "html.parser")
    
    try:
        captcha = soup_oferta.find("div", attrs= attrs_ad).text        
        if captcha == "Hacer clic para comprobarReintentar":
            break
    except:
        pass

    try:
        attrs_tit= {"id": "prefijoPuesto"}
        titulo = soup_oferta.find("div", class_= "heading-addons").find("h1", attrs= attrs_tit).text
    except:
        titulo = np.nan
    try:
        empresa = soup_oferta.find("div", class_= "heading-addons").find("a", class_= "link").text
    except:
        empresa = np.nan
    try:
        fecha = soup_oferta.find("div", class_= "row-matrioska").find("span", class_= "marked").text
    except:
        try:
            lista_bullets = soup_oferta.find("div", class_= "row-matrioska").find("ul", class_= "list-bullet-default").find_all("li")
            for bullet in lista_bullets:
                tag_bullet = bullet.text.strip()[:9]
                if tag_bullet == "Publicada":
                    fecha = bullet.text.strip()[12:]
        except:
            fecha = np.nan
        
    attrs_ubic= {"id": "prefijoPoblacion"}
    try:
        ubicacion = soup_oferta.find("div", class_= "row-matrioska").find("span", attrs= attrs_ubic).text.replace(",", "").strip()
    except:
        ubicacion = np.nan
        
    attrs_descr= {"id": "prefijoDescripcion1"}
    try:
        descripcion = soup_oferta.find("div", attrs= attrs_descr).text.replace("\n", "").strip()
    except:
        descripcion = np.nan
    try:
        lista_li = soup_oferta.find("div", class_= "row-matrioska").find_all("li")        
    
        for li in lista_li:

            tag = li.find("span").text[:7]

            if tag == "Salario":
                salario = li.find("span").text[9:]

            elif tag == "Experie":
                experiencia = li.find("span").text[20:]

            elif tag == "Tipo de":

                tipo_contrato = li.find("span").text[18:].split(",")[0]

                jornada = li.find("span").text[18:].split(",")[-1]
    except:
        salario = np.nan
        experiencia = np.nan
        tipo_contrato = np.nan
        jornada = np.nan
        
    try:
        lista_herramientas = soup_oferta.find("div", class_= "inner-expanded").find("ul", class_= "list-default").find_all("a")
        herramientas = [herramienta.text for herramienta in lista_herramientas]
    except:
        herramientas = np.nan
        
    try:
        funciones = soup_oferta.find("div", class_= "border-top").find("ul").find("span", class_= "list-default-text").text
    except:
        funciones = np.nan
    
    datos_ofertas_empleo.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, funciones, jornada, experiencia, tipo_contrato, salario, fecha_scrapeo, empleo, portal])
    contador += 1
    
    if contador % 100 == 0:
        sleep(20)

In [13]:
df_infojobs = pd.DataFrame(datos_ofertas_empleo, columns= ["titulo", "empresa", "fecha", "herramientas", "descripcion", "ubicacion", "funciones", "jornada", "experiencia", "tipo_contrato", "salario", "fecha_scrapeo", "url", "portal"])

In [15]:
df_infojobs = df_infojobs.dropna(how= "all")

In [18]:
# Lo guardo en la ruta de datos:
fecha = datetime.datetime.now().strftime("%d-%m-%y")
ruta_datos = "C:\\Users\\regue\\Desktop\\Data Science Projects\\PROJECTS\\IT_Job_Spain_Project\\Datos\\datos_sin_procesar\\"
nombre_archivo = f"datos_infojobs_{fecha}.csv"
ruta_completa = os.path.join(ruta_datos, nombre_archivo)

df_infojobs.to_csv(ruta_completa, index=False, sep=",")